In [ ]:
class DatabaseConnection:
    def __init__(self):
        pass
    
    def connection_properties(self, environment):
        if environment == 'production':
            print(prod)
        else:
            pass
        
        jdbcurl = "jdbc:sqlserver://{}:{};database={};authentication=ActiveDirectoryServicePrincipal;encrypt=true;trustServerCertificate=false;HostNameInCertificate=*.database.windows.net;loginTimeout=30;AADSecurePrincipalId={}@{};AADSecurePrincipalSecret={}".format(
            host,
            port,
            database,
            client_id,
            tenant_id,
            client_secret
        )
        
        connectionProperties = {
            "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
        }
        
        return {"url": jdbcurl, "properties": connectionProperties}
    
    def read_data_sqldb(self, read_query, environment):
        db_connection = self.connection_properties(environment)
        try:
            print("Reading query...")
            df_read_query = spark.read.jdbc(
                url=db_connection["url"],
                table=f"({read_query}) AS temp_table",
                properties=db_connection["properties"]
            )
            return df_read_query
        except Exception as exe:
            print("Running exception")
            error = type(exe).__name__
            error_des = str(exe)
            print(f"Error: {error}")
            print(f"Error Description: {error_des}")
            return None

# Usage
conn = DatabaseConnection()
df = conn.read_data_sqldb("SELECT 'test-a1' AS name1, 'test-a2' AS name2", "nonprod")

In [ ]:
def write_data_sqldb(self, dataframe, table_name, schema_name, environment, mode="append"):
    """
    Write dataframe to SQL Server table
    
    Args:
        dataframe: Spark DataFrame to write
        table_name: Target table name
        schema_name: Target schema name  
        environment: 'prod' or 'nonprod'
        mode: 'append', 'overwrite', 'ignore', 'error'
    """
    db_connection = self.connection_properties(environment)
    
    try:
        print(f"Writing data to [{schema_name}].[{table_name}]...")
        
        # Full table name with schema
        full_table_name = f"[{schema_name}].[{table_name}]"
        
        dataframe.write.jdbc(
            url=db_connection["url"],
            table=full_table_name,
            mode=mode,
            properties=db_connection["properties"]
        )
        
        print("Data written successfully!")
        return True
        
    except Exception as exe:
        print("Error occurred during write operation")
        error = type(exe).__name__
        error_des = str(exe)
        print(f"Error: {error}")
        print(f"Error Description: {error_des}")
        return False

In [ ]:
# Example usage
conn = DatabaseConnection()

# Assuming you have a dataframe 'df_to_insert'
success = conn.write_data_sqldb(
    dataframe=df_to_insert,
    table_name="MODEL_OUTPUT",
    schema_name="dm_db, 
    environment='nonprod',
    mode="append"  # or "overwrite"
)

In [ ]:
def write_data_sqldb_advanced(self, dataframe, table_name, schema_name, environment, 
                             mode="append", batch_size=1000, truncate_table=False):
    """
    Advanced write with more control options
    """
    db_connection = self.connection_properties(environment)
    
    try:
        print(f"Writing {dataframe.count()} rows to [{schema_name}].[{table_name}]...")
        
        full_table_name = f"[{schema_name}].[{table_name}]"
        
        # Additional properties for better performance
        write_properties = db_connection["properties"].copy()
        write_properties.update({
            "batchsize": str(batch_size),
            "truncate": str(truncate_table).lower()
        })
        
        dataframe.write.jdbc(
            url=db_connection["url"],
            table=full_table_name,
            mode=mode,
            properties=write_properties
        )
        
        print("Data written successfully!")
        return True
        
    except Exception as exe:
        print("Error occurred during write operation")
        error = type(exe).__name__
        error_des = str(exe)
        print(f"Error: {error}")
        print(f"Error Description: {error_des}")
        return False

In [ ]:
conn = DatabaseConnection()

# Example 1: Append data to existing table
success = conn.write_data_sqldb(
    dataframe=your_dataframe,
    table_name="MODEL_OUTPUT",
    schema_name="dm_db",
    environment='nonprod',
    mode="append"
)

# Example 2: Overwrite existing data
success = conn.write_data_sqldb(
    dataframe=your_dataframe,
    table_name="MODEL_OUTPUT",
    schema_name="dm_db",
    environment='nonprod',
    mode="overwrite"
)

# Example 3: Create sample data and insert
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Create sample dataframe (adjust schema to match your table)
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("value", DoubleType(), True)
])

sample_data = [(1, "test1", 10.5), (2, "test2", 20.3)]
sample_df = spark.createDataFrame(sample_data, schema)

# Insert sample data
success = conn.write_data_sqldb(
    dataframe=sample_df,
    table_name="MODEL_OUTPUT",
    schema_name="dm_db",
    environment='nonprod',
    mode="append"
)